# Table Question Answering

Given a bank statement table or any relational data stored in a structured format in a database, the user should be able to chat and ask questions from the data.

For example, consider a sample bank statement (assume it is present in a readable format). The following sample queries could be asked by the user:

a) **How much is my spend on online shopping?**
  - *Expected answer:* Should fetch spends from e-commerce websites such as Amazon, Myntra, etc.

b) **How much money have I spent on food and travel?**
  - *Expected answer:* Should fetch items from payments at hotels, restaurants, online food delivery portals, etc.

## Solutions

### Solution 1
1. Segregate table columns into two sets:
  - a. Direct inbuilt filters
  - b. Custom filters
2. Design custom filters.
3. Generate SQL queries corresponding to each query using an LLM by providing the table schema and available custom filter functions.
4. Run the generated SQL query on the table.

### Solution 2
1. Instead of using SQL language, generate Python queries using pandas.

### Solution 2 in Detail

1. **Segregate TABLE columns into two sets:**
  1. **Read table document and collect information corresponding to each column using preprocessing Python code:**
    - Collect each column's information including column name, data type, some sample values, and unique values.
    - Categorize the columns into two sets either manually or by asking an LLM:
      1. **direct_columns:** Columns where direct filters will be applied.
      2. **custom_columns:** Columns that require custom functions to filter.

2. **Design Custom Filters:**
  1. **Design a filter function "custom_filter" for custom_columns:**
    - The function takes three arguments: dataframe, column_name, and filter_value to filter rows. Example:
    ```python
    def custom_filter(df, description, filter_value):
        df = df[df['description'].str.contains(filter_value)]
        return df
    ```
  2. **Custom filter design is dependent on use cases:**
    1. **Filtering text by keyword search (e.g., regex, BM25):**
        - Applicable when columns contain some small description about products, like "high neckline dress in blue color made of cotton."
    2. **Filtering text by semantic meaning using an embedding model or LLM:**
      1. Applicable for our problem statement. This step is the main bottleneck for our problem statement.
      2. Apply the embedding model on filter_value and description entry to check whether both are related based on similarity score.
        - ```python
          score = embed(filter_value) @ embed(df[custom_column][index])
          ```
        - Take if score > threshold else reject.
      3. In some cases, embedding models are not good for comparing, especially for our case (e.g., most embedding models will fail to compare "Flipkart" to "Shopping"). In these cases, we need to fine-tune embedding models.
      4. Or we need a lightweight LLM that has good world knowledge to tell whether these two are related or not.
        - **Create prompt for LLM to filter, example prompt:**
        ```markdown
        You are provided sub-query and search database.
        Your task is to select ids from database which are related to sub-query. Output should be list of ids without any explanation.
        if no ids found, return empty list.
        ### Sub-query:
        food
        ### Database:
        {0: '550274051211 CHB', 1: 'CALL REF NO 3442 FROM A/C 22222222', 2: 'Amazon', 3: 'Tebay Trading Co', 4: 'Morrisons Petrol', 5: 'Business Loan', 6: 'James White Media', 7: 'ATM High Street', 8: 'Acom Advertising Studies', 9: 'Marriott Hotel'}
        ```
      5. If we know the range of filter_values in advance, we can map custom columns to dummy columns using embedding models or LLM. This will save a lot of time to filter.

3. **Generate Python pandas queries corresponding to the query using LLM by giving table schema and custom filter functions available:**
  1. **Create prompt for LLM which includes the table schema and custom filter function schema, and instructions to generate code:**
    - The table schema contains column name, data type, and some short text about the purpose of each column.
    - The filter function schema contains when to call this function and arguments to pass.

  **Example prompt:**
  ```markdown
  You are a chatbot designed to interact with a table to fetch information based on user questions. The table has the following columns with their data types:
  - Date: Text
  - Type: Text
  - Description: Text
  - Paid In: Float
  - Paid Out: Float
  - Balance: Float

  Based on the above structure, when a user asks a question, generate an appropriate pandas query code to fetch the relevant information from the database. Separate each filter code with a new line.
  For the column "Description", call a function "custom_filter(data_frame: pandas dataframe, column_name: here Description, sub_query: related sub-query)".
  Example:
  ### Query:
  How much is my spend on online shopping?
  ### Generated code:
  ``
  df1 = df[df['Paid Out'] > 0]
  df2 = custom_filter(df1, 'Description', 'online shopping')
  amount = df2['Paid Out'].sum()
  print("Amount Spent: ", amount)
  ``
  Don't write 'read_csv' and 'custom_filter' code by yourself, it's already there.
  ### Here is the query:
  {query}
  ```

4. **Parse and run this generated code using some text code executer like "ast".**
```



## Step 1: Data Analysis to segregate TABLE columns into two sets.

In [ ]:
import pandas as pd
import random

# Read the CSV file
df = pd.read_csv('2024-06-29_21-33_table.csv',encoding='unicode_escape')

# Iterate over each column
for column in df.columns:

    # Get the unique values in the column
    unique = df[column].unique()
    unique = pd.Series(unique).dropna().unique()

    if len(unique)>5:
      unique_values = random.sample(list(unique),3)
    else:
      unique_values = list(unique)
    dtype= 'str'

    try:
      x=[int(i) for i in unique if i]
      dtype ='int'
    except:
      pass
    # Print the column name, data type, and unique values
    print(f"Column: {column}")
    print(f"Data Type: {dtype}")
    print(f"Example Values: {unique_values}")
    print()

Column: Date
Data Type: str
Example Values: ['01 Dec 2014', '24 Oct 2014', '01 Nov 2014']

Column: Type
Data Type: str
Example Values: ['DIGITAL BANKING', 'Int Bank', 'BACS']

Column: Description
Data Type: str
Example Values: ['James White Media', '550274051211 CHB', 'Various Payment']

Column: Paid In
Data Type: int
Example Values: [20000.0, 9.33]

Column: Paid Out
Data Type: int
Example Values: [280.0, 515.22, 190.4]

Column: Balance
Data Type: int
Example Values: [1613.5, 16535.21, 18034.43]



## Step 2: Design Custom filter function

In [ ]:
import json
def custom_filter_(data_frame, column_name, sub_query, **kwargs):
    database = data_frame[column_name].dropna().unique()
    batch_size = 10
    ids_final = []

    for i in range(0, len(database), batch_size):
        batch = database[i:i + batch_size]
        database_dict = {i + idx: item for idx, item in enumerate(batch)}
        prompt_template = kwargs['prompt_template']
        prompt = prompt_template.format(sub_query=sub_query, database=str(database_dict))
        #print(prompt)
        api_response = get_completion(
            [{"role": "user", "content": prompt}],
            model="gpt-4"
        )

        ids = api_response.choices[0].message.content
        try:
            ids = json.loads(ids)
        except json.JSONDecodeError:
            ids = []
        #print(ids)
        ids_final.extend(ids)

    filtered_list = database[ids_final]
    filtered_data_frame = data_frame[data_frame[column_name].isin(filtered_list)]

    print(f"\nList of transactions for {sub_query}:")
    for col in filtered_data_frame.columns:
        print(f"{col:<15}", end=' ')
    print()

    for index, row in filtered_data_frame.iterrows():
        for col in row:
            print(f"{col:<15}", end=' ')
        print()

    print('\n\n')
    return filtered_data_frame



In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


## Step 3 & 4: Generate Python pandas queries corresponding to the query using LLM and run.

In [ ]:
import ast

PROMPT1 = """
You are a chatbot designed to interact with a table to fetch information based on user questions. The table contains has following columns with thier datatype:
Date: Text, Type: Text, Description: Text, Paid In: Float, Paid Out: Float, Balance: Float

Based on the above structure, when a user asks a question, generate an appropriate pandas query code to fetch the relevant information from the database. Separate each filter code with a new line.
For column "Description" call a funtion "custom_filter(data_frame: pandas dataframe, column_name: here Description, sub_query: related sub-query)".
  example:
    query:
      How much is my spend on online shopping?
    generated code:
      ```python
         df1 = df[df['Paid Out]'>0]
         df2 = custom_filter(df1, 'Description', 'online shoping')
         amount = df2['Paid Out'].sum()
         print("Amount Spend: ",amount)
      ```

Don't write 'read_csv' and 'custom_filter' code by yourself, it's already there.

### Here is the query:
    {query}

    """

PROMPT2="""You are provided sub-query and search database.
           Your task is to select ids from database which are related to sub-query. Output should be list of ids without any explanation.
           if no ids found, return empty list.
           ### Sub-query:
               {sub_query}
           ### Database:
               {database}
           """


In [ ]:
from openai import OpenAI
import os

api_key = "sk-.."
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", api_key))


def get_completion(
    messages: list[dict[str, str]],
    model: str = "gpt-4",
    max_tokens=500,
    temperature=0,
    stop=None,
    seed=123,
    tools=None,
    logprobs=None,
    top_logprobs=None,
) -> str:
    params = {
        "model": model,
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "stop": stop,
        "seed": seed,
        "logprobs": logprobs,
        "top_logprobs": top_logprobs,
    }
    if tools:
        params["tools"] = tools

    completion = client.chat.completions.create(**params)
    return completion

In [ ]:
import re
import ast
from functools import partial

queries= ["How much money i have spent on food and travel",
          "How much is my spend on online shopping",
          "How much is my balance now",
          "How much balance got credited in my account",
          "My spending on Hotels only"]

for query in queries[:]:
  print("*"*100,'\n')
  print(f"Query Asked: {query}\n")
  custom_filter= partial(custom_filter_, query=query, prompt_template=PROMPT2)
  prompt= PROMPT1.format(query=query)
  api_response = get_completion(
          [{"role": "user", "content": prompt}],
          model="gpt-4",
      )
  response = api_response.choices[0].message.content


  pattern = re.compile(r'```python\n(.*?)```', re.DOTALL)
  match = pattern.search(response)
  if match:
      code = match.group(1)
      print(f"Generated code:\n{code}\n")
      tree = ast.parse(code)
      compiled = compile(tree, filename="<ast>", mode="exec")
      exec(compiled, None, None)
  else:
      print("No code found in the response.")


**************************************************************************************************** 

Query Asked: How much money i have spent on food and travel

Generated code:
df1 = df[df['Paid Out']>0]
df2 = custom_filter(df1, 'Description', 'food')
df3 = custom_filter(df1, 'Description', 'travel')
amount = df2['Paid Out'].sum() + df3['Paid Out'].sum()
print("Amount Spent: ", amount)



List of transactions for food:
Date            Type            Description     Paid In         Paid Out        Balance         




List of transactions for travel:
Date            Type            Description     Paid In         Paid Out        Balance         
01 Nov 2014     BACS            Marriot Hotel   nan             177.0           18034.43        
01 Nov 2014     Fastor Payment  Abellio Scotrail Ltd nan             122.22          17857.43        



Amount Spent:  299.22
**************************************************************************************************** 

Query Asked: How

## Final Discussion:

1. Since I don't have GPUs to run open-source LLMs like Mistral and LLaMA, I used the GPT-4 API.

2. Due to time constraints and accuracy concerns, I extracted "custom columns" like "Description" manually instead of using an LLM to decide them. Utilizing an LLM would make it more adaptable for any table. I also prepared prompts manually, which can also be automated using an LLM.

3. The main challenge was associating the query with the description columns.
   1. I tried embedding models like 'all-minilm-l6-v2' and 'BAAI/bge-m3', but these models are trained to capture similar meaning sentences, not for entity relationships in a knowledge graph, so their performance was poor.
   2. I searched for knowledge graph embedding models but couldn't find any suitable ones.
   3. Due to limited time, I experimented with filtering using GPT-4, which worked well. GPT-4 has good world knowledge, such as knowing that "Abellio ScotRail" is a train company and associating it with "travel."
   4. I applied GPT-4 to unique values of the "Description" column in batches of 10 (i.e., 1 LLM call would compare the subquery 'travel' with 10 description values).
   5. LLM calls are not scalable and can be costly for 10k unique values. The best approach would be to fine-tune embedding models on data that contain entity relationships as well as similar sentences.

4. Since I know basic SQL, I chose to generate queries in Python instead of SQL so that I could debug things easily. However, for production, we need to implement it in SQL.

5. This is a basic demo. To handle all limitations and make it production-ready, we need time to conduct R&D on each component of the system.

6. There might be other approaches to achieve this, which I haven't explored.